In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)

# BATSMAN SELECTION

In [2]:
batsman_df = pd.read_excel("batsman r2.xlsx")
batsman_df.head()

,match id,batsman,batsman_id,batsman_dob,batsman_details,is_batsman_captain,is_batsman_keeper,inning,runs,balls_faced,over_faced_first,wicket kind,out_by_bowler,out_by_fielder,bowler_id,bowler_dob,bowler_details,is_bowler_keeper,is_bowler_captain,strike_rate,Fours,Sixes,match_dt,ingestion_timestamp
0,8587795,AJ Fh,37351,1986-11-17,AUS:Right-hand bat:Slow left-arm orthodox:,1,0,2,14,15,1.3,lbw,WA Ar,None,6718844.0,1997-02-05,AUS:Right-hand bat:Right-arm fast:,0.0,0.0,93.33,1.0,NaN,2021-01-08,2022-03-22 12:43:21
1,8587795,Md Ni,181404,1985-01-01,AFG:Right-hand bat:Right-arm offbreak:,0,0,2,71,41,6.3,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,173.17,9.0,2.0,2021-01-08,2022-03-22 12:43:21
2,8587795,JW Ws,1635773,1988-08-13,AUS:Right-hand bat:Right-arm medium:,0,0,1,16,13,17.2,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,123.08,1.0,NaN,2021-01-08,2022-03-22 12:43:21
3,8587795,DR Bs,2170762,1991-04-30,ENG:Right-hand bat:Slow left-arm orthodox:,0,0,1,2,4,19.6,caught,KW Rn,Md Ni,1905847.0,1991-02-12,AUS:Right-hand bat:Right-arm fast-medium:,0.0,0.0,50.00,NaN,NaN,2021-01-08,2022-03-22 12:43:21
4,8587795,AT Cy,2285051,1991-08-27,AUS:Left-hand bat:None:,1,1,1,42,25,8.2,caught,Id Wm,JA Pe,1594319.0,1988-12-18,PAK:Left-hand bat:Slow left-arm orthodox:,0.0,0.0,168.00,3.0,3.0,2021-01-08,2022-03-22 12:43:21


In [3]:
def calculate_metrics(batsman_df):
    batsman_df['half_centuries'] = (batsman_df['runs'] >= 50).astype(int)
    batsman_df['centuries'] = (batsman_df['runs'] >= 100).astype(int)
    
    batsman_stats = batsman_df.groupby('batsman_id').agg({
        'runs': 'sum',
        'half_centuries': 'sum',
        'centuries': 'sum',
        'match id': 'nunique',
        'strike_rate': 'mean'
    }).reset_index()
    batsman_stats.columns = ['batsman_id', 'cumulative_runs', 'half_centuries','centuries','matches','strike_rate']
    
    batsman_stats['average'] = batsman_stats['cumulative_runs'] / batsman_stats['matches']
    
    
    
    #scores assignment
    
    batsman_stats['score_strike'] = np.where(batsman_stats['strike_rate'] >= 150, 50,
                                      np.where((batsman_stats['strike_rate'] >= 100) & (batsman_stats['strike_rate'] < 150), 40,
                                               np.where((batsman_stats['strike_rate'] >= 80) & (batsman_stats['strike_rate'] < 100), 30,
                                                        0)))
    batsman_stats['score_half_centuries'] = np.where(batsman_stats['half_centuries'] >= 5, 20,
                                                     np.where((batsman_stats['half_centuries'] == 3) | (batsman_stats['half_centuries'] == 4), 10,
                                                              np.where((batsman_stats['half_centuries'] == 1) | (batsman_stats['half_centuries'] == 2), 5,
                                                                       0)))
    batsman_stats['score_centuries'] = np.where(batsman_stats['centuries'] >= 3, 30,
                                                     np.where((batsman_stats['centuries'] == 2), 20,
                                                              np.where((batsman_stats['centuries'] == 1), 10,
                                                                       0)))
    
    batsman_stats['score_average'] = np.where(batsman_stats['average'] >= 50, 30,
                                              np.where((batsman_stats['average'] >= 40) & (batsman_stats['average'] < 50), 20,
                                                       np.where((batsman_stats['average'] >= 30) & (batsman_stats['average'] < 40), 10,
                                                                5)))
    
    
    batsman_stats['total_score'] = batsman_stats['score_strike'] + batsman_stats['score_half_centuries'] + batsman_stats['score_centuries'] + batsman_stats['score_average']
    
    return batsman_stats

batsman_stats = calculate_metrics(batsman_df)
batsman_stats

,batsman_id,cumulative_runs,half_centuries,centuries,matches,strike_rate,average,score_strike,score_half_centuries,score_centuries,score_average,total_score
0,31464,10,0,0,3,40.910000,3.333333,0,0,0,5,5
1,34061,729,2,0,52,119.160577,14.019231,40,5,0,5,50
2,36665,208,0,0,10,99.854000,20.800000,30,0,0,5,35
3,37351,1517,12,0,65,101.781692,23.338462,40,20,0,5,65
4,41740,976,6,0,41,116.654878,23.804878,40,20,0,5,65
...,...,...,...,...,...,...,...,...,...,...,...,...
2058,9822322,0,0,0,1,0.000000,0.000000,0,0,0,5,5
2059,9822392,0,0,0,1,0.000000,0.000000,0,0,0,5,5
2060,9874052,42,0,0,1,210.000000,42.000000,50,0,0,20,70
2061,9883250,20,0,0,1,80.000000,20.000000,30,0,0,5,35


In [4]:
selected_batsman = batsman_stats[batsman_stats['cumulative_runs'] > 100]
selected_batsman

,batsman_id,cumulative_runs,half_centuries,centuries,matches,strike_rate,average,score_strike,score_half_centuries,score_centuries,score_average,total_score
1,34061,729,2,0,52,119.160577,14.019231,40,5,0,5,50
2,36665,208,0,0,10,99.854000,20.800000,30,0,0,5,35
3,37351,1517,12,0,65,101.781692,23.338462,40,20,0,5,65
4,41740,976,6,0,41,116.654878,23.804878,40,20,0,5,65
5,46794,564,6,0,16,111.088750,35.250000,40,20,0,10,70
...,...,...,...,...,...,...,...,...,...,...,...,...
1974,9009265,111,0,0,7,133.374286,15.857143,40,0,0,5,45
1983,9036516,331,2,1,12,104.068333,27.583333,40,5,10,5,60
1987,9047499,162,0,0,7,78.822857,23.142857,0,0,0,5,5
2001,9098571,124,2,0,5,148.110000,24.800000,40,5,0,5,50


In [5]:
nearbest_11 = pd.read_csv("nearbest_11.csv")

In [6]:
# calculate consistency
def calculate_consistency_batsman(df, batsman_id):
    batsman_df = df[df['batsman_id'] == batsman_id]

    # Calculate the standard deviation of strike rate
    std_sr = batsman_df['strike_rate'].std()
    
    consistency = {
        'batsman_id': batsman_id,
        'std_strike_rate': std_sr
    }
    
    return consistency

In [7]:
def calculate_consistency_for_all_batsman(df,df_for_id):
    # Get a list of unique batsman IDs
    batsman_ids = df_for_id['batsman_id']
    
    # Initialize an empty list to store the results
    results = []
    
    # Iterate over each batsman ID and calculate consistency
    for batsman_id in batsman_ids:
        consistency = calculate_consistency_batsman(df, batsman_id)
        results.append(consistency)
    
    # Convert the results list to a DataFrame
    consistency_df = pd.DataFrame(results)
    
    return consistency_df

consistency_df = calculate_consistency_for_all_batsman(batsman_df, selected_batsman)
print(consistency_df)

     batsman_id  std_strike_rate
0         34061        87.348364
1         36665        52.793254
2         37351        52.536959
3         41740        53.823271
4         46794        56.299157
..          ...              ...
781     9009265        46.872057
782     9036516        65.331280
783     9047499        42.086778
784     9098571       102.602814
785     9320597        56.759101

[786 rows x 2 columns]


In [8]:
# calculate recency
def calculate_recency_and_performance(df, batsman_id, recent_matches=5):
    # Filter data for the specific batsman
    batsman_df = df[df['batsman_id'] == batsman_id]
    
    # Convert match dates to datetime format
    batsman_df['match_dt'] = pd.to_datetime(batsman_df['match_dt'])
    
    # Sort the matches by date in descending order to get the most recent matches
    batsman_df = batsman_df.sort_values(by='match_dt', ascending=False)
    
    # Select the most recent matches
    recent_batsman_df = batsman_df.head(recent_matches)
    
    # Calculate recency as the mean of the timestamps of the recent matches
    match_dates = recent_batsman_df['match_dt'].apply(lambda x: x.timestamp())
    recency_score = np.mean(match_dates)
    
    # Summarize performance metrics in recent matches
    total_runs = recent_batsman_df['runs'].sum()
    total_balls_faced = recent_batsman_df['balls_faced'].sum()
    matches_played = len(recent_batsman_df)
    recent_sr = total_runs / total_balls_faced *100 if total_balls_faced > 0 else 0

    
    recency = {
        'batsman_id': batsman_id,
        'recency_score': recency_score,
        'matches_played': matches_played,
        'total_runs': total_runs,
        'total_balls_faced': total_balls_faced,
        'recent_sr': recent_sr
    }
    
    return recency

In [9]:
def calculate_recency_for_all_batsman(df,df_for_id):
    # Get a list of unique batsman IDs
    batsman_ids = df_for_id['batsman_id']
    
    results = []
    
    # Iterate over each batsman ID and calculate consistency
    for batsman_id in batsman_ids:
        recency = calculate_recency_and_performance(df, batsman_id)
        results.append(recency)
    
    # Convert the results list to a DataFrame
    recency_df = pd.DataFrame(results)
    
    return recency_df

recency_df = calculate_recency_for_all_batsman(batsman_df, selected_batsman)
print(recency_df)

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_13712\3194042368.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batsman_df['match_dt'] = pd.to_datetime(batsman_df['match_dt'])
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_13712\3194042368.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batsman_df['match_dt'] = pd.to_datetime(batsman_df['match_dt'])
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_13712\3194042368.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFra

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_13712\3194042368.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batsman_df['match_dt'] = pd.to_datetime(batsman_df['match_dt'])
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_13712\3194042368.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batsman_df['match_dt'] = pd.to_datetime(batsman_df['match_dt'])
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_13712\3194042368.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFra

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_13712\3194042368.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batsman_df['match_dt'] = pd.to_datetime(batsman_df['match_dt'])
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_13712\3194042368.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batsman_df['match_dt'] = pd.to_datetime(batsman_df['match_dt'])
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_13712\3194042368.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFra

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_13712\3194042368.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batsman_df['match_dt'] = pd.to_datetime(batsman_df['match_dt'])
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_13712\3194042368.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batsman_df['match_dt'] = pd.to_datetime(batsman_df['match_dt'])
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_13712\3194042368.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFra

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_13712\3194042368.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batsman_df['match_dt'] = pd.to_datetime(batsman_df['match_dt'])
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_13712\3194042368.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batsman_df['match_dt'] = pd.to_datetime(batsman_df['match_dt'])
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_13712\3194042368.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFra

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_13712\3194042368.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batsman_df['match_dt'] = pd.to_datetime(batsman_df['match_dt'])
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_13712\3194042368.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batsman_df['match_dt'] = pd.to_datetime(batsman_df['match_dt'])
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_13712\3194042368.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFra

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_13712\3194042368.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batsman_df['match_dt'] = pd.to_datetime(batsman_df['match_dt'])
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_13712\3194042368.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batsman_df['match_dt'] = pd.to_datetime(batsman_df['match_dt'])
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_13712\3194042368.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFra

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_13712\3194042368.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batsman_df['match_dt'] = pd.to_datetime(batsman_df['match_dt'])
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_13712\3194042368.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batsman_df['match_dt'] = pd.to_datetime(batsman_df['match_dt'])
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_13712\3194042368.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFra

     batsman_id  recency_score  matches_played  total_runs  total_balls_faced  \
0         34061   1.673914e+09               5          38                 32   
1         36665   1.611446e+09               5         158                118   
2         37351   1.685992e+09               5         128                125   
3         41740   1.680376e+09               5         116                102   
4         46794   1.685837e+09               5         230                153   
..          ...            ...             ...         ...                ...   
781     9009265   1.679063e+09               5          84                 60   
782     9036516   1.678614e+09               5         200                112   
783     9047499   1.691816e+09               5          77                 82   
784     9098571   1.684904e+09               5         124                 70   
785     9320597   1.685802e+09               5          91                 65   

      recent_sr  
0    118.

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_13712\3194042368.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batsman_df['match_dt'] = pd.to_datetime(batsman_df['match_dt'])
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_13712\3194042368.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batsman_df['match_dt'] = pd.to_datetime(batsman_df['match_dt'])
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_13712\3194042368.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFra

In [10]:
c_and_r_batsman  =pd.merge(consistency_df, recency_df, on ='batsman_id')
c_and_r_batsman

,batsman_id,std_strike_rate,recency_score,matches_played,total_runs,total_balls_faced,recent_sr
0,34061,87.348364,1.673914e+09,5,38,32,118.750000
1,36665,52.793254,1.611446e+09,5,158,118,133.898305
2,37351,52.536959,1.685992e+09,5,128,125,102.400000
3,41740,53.823271,1.680376e+09,5,116,102,113.725490
4,46794,56.299157,1.685837e+09,5,230,153,150.326797
...,...,...,...,...,...,...,...
781,9009265,46.872057,1.679063e+09,5,84,60,140.000000
782,9036516,65.331280,1.678614e+09,5,200,112,178.571429
783,9047499,42.086778,1.691816e+09,5,77,82,93.902439
784,9098571,102.602814,1.684904e+09,5,124,70,177.142857


In [11]:
final_batsman  =pd.merge(c_and_r_batsman, selected_batsman, on ='batsman_id')
final_batsman

,batsman_id,std_strike_rate,recency_score,matches_played,total_runs,total_balls_faced,recent_sr,cumulative_runs,half_centuries,centuries,matches,strike_rate,average,score_strike,score_half_centuries,score_centuries,score_average,total_score
0,34061,87.348364,1.673914e+09,5,38,32,118.750000,729,2,0,52,119.160577,14.019231,40,5,0,5,50
1,36665,52.793254,1.611446e+09,5,158,118,133.898305,208,0,0,10,99.854000,20.800000,30,0,0,5,35
2,37351,52.536959,1.685992e+09,5,128,125,102.400000,1517,12,0,65,101.781692,23.338462,40,20,0,5,65
3,41740,53.823271,1.680376e+09,5,116,102,113.725490,976,6,0,41,116.654878,23.804878,40,20,0,5,65
4,46794,56.299157,1.685837e+09,5,230,153,150.326797,564,6,0,16,111.088750,35.250000,40,20,0,10,70
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
781,9009265,46.872057,1.679063e+09,5,84,60,140.000000,111,0,0,7,133.374286,15.857143,40,0,0,5,45
782,9036516,65.331280,1.678614e+09,5,200,112,178.571429,331,2,1,12,104.068333,27.583333,40,5,10,5,60
783,9047499,42.086778,1.691816e+09,5,77,82,93.902439,162,0,0,7,78.822857,23.142857,0,0,0,5,5
784,9098571,102.602814,1.684904e+09,5,124,70,177.142857,124,2,0,5,148.110000,24.800000,40,5,0,5,50


In [12]:
low = final_batsman['recency_score'].max()
high = final_batsman['recency_score'].min()
mid = (low+high)/2
final_batsman2 = final_batsman[final_batsman['recency_score']>mid]

In [13]:
# Adding normalised columns
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
final_batsman2['total_score_normalised'] = scaler.fit_transform(final_batsman2[['total_score']])
final_batsman2['std_sr_normalised'] = scaler.fit_transform(-final_batsman2[['std_strike_rate']])
final_batsman2['recent_sr_normalised'] = scaler.fit_transform(final_batsman2[['recent_sr']])
final_batsman2['recent_matches_played'] = scaler.fit_transform(final_batsman2[['matches_played']])
final_batsman2['sum'] = final_batsman2['std_sr_normalised'] + final_batsman2['recent_sr_normalised'] + final_batsman2['total_score_normalised'] +final_batsman2['recent_matches_played']
final_batsman2

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_13712\117961587.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_batsman2['total_score_normalised'] = scaler.fit_transform(final_batsman2[['total_score']])
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_13712\117961587.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_batsman2['std_sr_normalised'] = scaler.fit_transform(-final_batsman2[['std_strike_rate']])
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_13712\117961587.py:6: SettingWithCopyWarning: 


,batsman_id,std_strike_rate,recency_score,matches_played,total_runs,total_balls_faced,recent_sr,cumulative_runs,half_centuries,centuries,matches,strike_rate,average,score_strike,score_half_centuries,score_centuries,score_average,total_score,total_score_normalised,std_sr_normalised,recent_sr_normalised,recent_matches_played,sum
0,34061,87.348364,1.673914e+09,5,38,32,118.750000,729,2,0,52,119.160577,14.019231,40,5,0,5,50,0.473684,0.574410,0.471347,1.0,2.519441
2,37351,52.536959,1.685992e+09,5,128,125,102.400000,1517,12,0,65,101.781692,23.338462,40,20,0,5,65,0.631579,0.775672,0.392642,1.0,2.799893
3,41740,53.823271,1.680376e+09,5,116,102,113.725490,976,6,0,41,116.654878,23.804878,40,20,0,5,65,0.631579,0.768236,0.447160,1.0,2.846975
4,46794,56.299157,1.685837e+09,5,230,153,150.326797,564,6,0,16,111.088750,35.250000,40,20,0,10,70,0.684211,0.753921,0.623350,1.0,3.061481
5,62432,62.318474,1.695082e+09,5,115,80,143.750000,1911,7,0,87,121.130345,21.965517,40,20,0,5,65,0.631579,0.719120,0.591691,1.0,2.942390
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
781,9009265,46.872057,1.679063e+09,5,84,60,140.000000,111,0,0,7,133.374286,15.857143,40,0,0,5,45,0.421053,0.808424,0.573639,1.0,2.803116
782,9036516,65.331280,1.678614e+09,5,200,112,178.571429,331,2,1,12,104.068333,27.583333,40,5,10,5,60,0.578947,0.701702,0.759312,1.0,3.039962
783,9047499,42.086778,1.691816e+09,5,77,82,93.902439,162,0,0,7,78.822857,23.142857,0,0,0,5,5,0.000000,0.836090,0.351737,1.0,2.187827
784,9098571,102.602814,1.684904e+09,5,124,70,177.142857,124,2,0,5,148.110000,24.800000,40,5,0,5,50,0.473684,0.486216,0.752436,1.0,2.712336


In [14]:
final_sorted = final_batsman2.sort_values(by=['sum'], ascending=[False])
final_sorted.head()

,batsman_id,std_strike_rate,recency_score,matches_played,total_runs,total_balls_faced,recent_sr,cumulative_runs,half_centuries,centuries,matches,strike_rate,average,score_strike,score_half_centuries,score_centuries,score_average,total_score,total_score_normalised,std_sr_normalised,recent_sr_normalised,recent_matches_played,sum
183,2275195,75.142067,1.702668e+09,5,191,99,192.929293,2480,17,3,95,126.458211,26.105263,40,20,30,5,95,0.947368,0.644981,0.828428,1.0,3.420777
164,2162782,58.786308,1.697103e+09,5,233,149,156.375839,3498,32,6,97,123.307835,36.061856,40,20,30,10,100,1.000000,0.739542,0.652468,1.0,3.392010
180,2263736,43.395586,1.680670e+09,5,179,134,133.582090,4031,40,2,92,115.673261,43.815217,40,20,20,20,100,1.000000,0.828523,0.542745,1.0,3.371268
102,1749075,73.266110,1.699885e+09,5,178,99,179.797980,3203,23,3,112,130.018214,28.598214,40,20,30,5,95,0.947368,0.655827,0.765217,1.0,3.368412
290,3125562,64.481831,1.701786e+09,5,201,130,154.615385,2551,23,4,80,136.978500,31.887500,40,20,30,10,100,1.000000,0.706613,0.643994,1.0,3.350607


In [15]:
# comparing with near best 11 provided
bat_best11_df =pd.merge(final_sorted, nearbest_11, left_on='batsman_id', right_on='Player_id', how='inner')
bat_best11_df

,batsman_id,std_strike_rate,recency_score,matches_played,total_runs,total_balls_faced,recent_sr,cumulative_runs,half_centuries,centuries,matches,strike_rate,average,score_strike,score_half_centuries,score_centuries,score_average,total_score,total_score_normalised,std_sr_normalised,recent_sr_normalised,recent_matches_played,sum,Player_id
0,2275195,75.142067,1.702668e+09,5,191,99,192.929293,2480,17,3,95,126.458211,26.105263,40,20,30,5,95,0.947368,0.644981,0.828428,1.0,3.420777,2275195
1,325814,59.241404,1.688291e+09,5,133,82,162.195122,822,8,1,27,134.665926,30.444444,40,20,10,10,80,0.789474,0.736911,0.680481,1.0,3.206865,325814
2,1626526,58.775486,1.697864e+09,5,189,140,135.000000,2737,18,1,90,127.727889,30.411111,40,20,10,10,80,0.789474,0.739604,0.549570,1.0,3.078648,1626526
3,82228,57.870937,1.686545e+09,5,183,122,150.000000,874,3,1,32,133.901875,27.312500,40,10,10,5,65,0.631579,0.744834,0.621777,1.0,2.998189,82228
4,1506098,61.534111,1.674380e+09,5,209,151,138.410596,677,4,1,22,108.984545,30.772727,40,10,10,10,70,0.684211,0.723655,0.565988,1.0,2.973854,1506098
5,4171460,43.931313,1.690917e+09,5,90,55,163.636364,400,0,0,27,103.976667,14.814815,40,0,0,5,45,0.421053,0.825426,0.687419,1.0,2.933897,4171460
6,8250115,44.223173,1.699799e+09,5,160,152,105.263158,700,6,0,21,100.807619,33.333333,40,20,0,10,70,0.684211,0.823739,0.406424,1.0,2.914374,8250115


# BOWLER SELECTION

In [16]:
bowler_df = pd.read_excel("bowler r2.xlsx")
bowler_df.head()

,match id,bowler,bowler_id,bowler_dob,bowler_details,is_bowler_captain,is_bowler_keeper,inning,runs,wicket_count,balls_bowled,economy,maiden,dots,Fours,Sixes,wides,noballs,match_dt,ingestion_timestamp
0,8587795,PM Se,55299,1984-11-25,AUS:Right-hand bat:Right-arm fast-medium:,0,0,2,38,0,24,9.50,0,8,4,1,1,0,2021-01-08,2022-03-21 03:44:58
1,8587795,Md Ni,181404,1985-01-01,AFG:Right-hand bat:Right-arm offbreak:,0,0,1,9,0,6,9.00,0,1,1,0,0,0,2021-01-08,2022-03-21 03:44:58
2,8587795,Id Wm,1594319,1988-12-18,PAK:Left-hand bat:Slow left-arm orthodox:,0,0,1,28,1,24,7.00,0,6,3,0,0,0,2021-01-08,2022-03-21 03:44:58
3,8587795,KW Rn,1905847,1991-02-12,AUS:Right-hand bat:Right-arm fast-medium:,0,0,1,40,1,24,10.00,0,10,2,3,3,0,2021-01-08,2022-03-21 03:44:58
4,8587795,DR Bs,2170762,1991-04-30,ENG:Right-hand bat:Slow left-arm orthodox:,0,0,2,37,1,24,9.25,0,7,4,1,0,0,2021-01-08,2022-03-21 03:44:58


In [17]:
def calculate_metrics_bowler(bowler_df):
    bowler_df['4w_per_inning'] = ((bowler_df['wicket_count'] == 4) & (bowler_df['inning'] == 1)).astype(int)
    bowler_stats = bowler_df.groupby('bowler_id').agg({
        'wicket_count': 'sum',
        'balls_bowled': 'sum',
        'runs': 'sum',
        '4w_per_inning': 'sum',
        'match id': 'nunique',
        'economy': 'mean'
    }).reset_index()
    bowler_stats.columns = ['bowler_id', 'cumulative_wickets', 'balls_bowled','runs_given','count_4w_per_inning','matches','economy']
    
    bowler_stats['strike_rate_bowl'] = (bowler_stats['balls_bowled'] / bowler_stats['cumulative_wickets'])
    bowler_stats['average_bowl'] = (bowler_stats['runs_given'] / bowler_stats['cumulative_wickets'])
    
    

    #scores assignment
    
    bowler_stats['score_strike_bowl'] = np.where(bowler_stats['strike_rate_bowl'] <= 15, 30,
                                      np.where((bowler_stats['strike_rate_bowl'] > 15) & (bowler_stats['strike_rate_bowl'] <= 19), 20,
                                               np.where((bowler_stats['strike_rate_bowl'] > 19) & (bowler_stats['strike_rate_bowl'] <= 24), 10,
                                                        0)))

    bowler_stats['score_economy'] = np.where(bowler_stats['economy'] <= 3, 50,
                                              np.where((bowler_stats['economy'] > 3) & (bowler_stats['economy'] <= 5), 40,
                                                       np.where((bowler_stats['economy'] > 5) & (bowler_stats['economy'] < 7), 30,
                                                                0)))
    
    
    bowler_stats['score_average_bowl'] = np.where(bowler_stats['average_bowl'] <= 20, 30,
                                              np.where((bowler_stats['average_bowl'] > 20) & (bowler_stats['average_bowl'] <= 30), 20,
                                                       np.where((bowler_stats['average_bowl'] > 30) & (bowler_stats['average_bowl'] <= 40), 10,
                                                                0)))
    bowler_stats['score_4w'] = np.where(bowler_stats['count_4w_per_inning'] >= 4, 30,
                                              np.where((bowler_stats['count_4w_per_inning'] == 2) | (bowler_stats['count_4w_per_inning'] == 3), 20,
                                                       np.where((bowler_stats['count_4w_per_inning'] == 1), 10,
                                                                0)))
    
    
    bowler_stats['total_score_bowl'] = bowler_stats['score_strike_bowl'] + bowler_stats['score_economy'] + bowler_stats['score_average_bowl'] + bowler_stats['score_4w']
    
    return bowler_stats

bowler_stats = calculate_metrics_bowler(bowler_df)
bowler_stats

,bowler_id,cumulative_wickets,balls_bowled,runs_given,count_4w_per_inning,matches,economy,strike_rate_bowl,average_bowl,score_strike_bowl,score_economy,score_average_bowl,score_4w,total_score_bowl
0,34061,27,635,965,0,46,9.490000,23.518519,35.740741,10,0,10,0,20
1,41740,5,90,76,0,10,5.900000,18.000000,15.200000,20,30,30,0,80
2,49496,31,516,533,0,23,6.203043,16.645161,17.193548,20,30,30,0,80
3,55299,46,709,978,0,35,8.097429,15.413043,21.260870,20,0,20,0,40
4,62432,72,1178,1418,0,77,7.826494,16.361111,19.694444,20,0,30,0,50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1506,9822322,0,24,29,0,2,7.250000,inf,inf,0,0,0,0,0
1507,9822392,3,18,7,0,1,2.330000,6.000000,2.333333,30,50,30,0,110
1508,9822420,1,12,34,0,1,17.000000,12.000000,34.000000,30,0,10,0,40
1509,9822441,2,24,57,0,1,14.250000,12.000000,28.500000,30,0,20,0,50


In [18]:
selected_bowler = bowler_stats[bowler_stats['cumulative_wickets'] > 10]
selected_bowler

,bowler_id,cumulative_wickets,balls_bowled,runs_given,count_4w_per_inning,matches,economy,strike_rate_bowl,average_bowl,score_strike_bowl,score_economy,score_average_bowl,score_4w,total_score_bowl
0,34061,27,635,965,0,46,9.490000,23.518519,35.740741,10,0,10,0,20
2,49496,31,516,533,0,23,6.203043,16.645161,17.193548,20,30,30,0,80
3,55299,46,709,978,0,35,8.097429,15.413043,21.260870,20,0,20,0,40
4,62432,72,1178,1418,0,77,7.826494,16.361111,19.694444,20,0,30,0,50
6,65183,16,189,280,1,9,9.090000,11.812500,17.500000,30,0,30,10,70
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1378,8833075,21,225,239,0,11,6.221818,10.714286,11.380952,30,30,30,0,90
1381,8845850,32,638,866,0,31,8.374194,19.937500,27.062500,10,0,20,0,30
1390,8907338,47,837,1141,1,41,8.232439,17.808511,24.276596,20,0,20,10,50
1408,8986193,22,283,338,0,13,7.282308,12.863636,15.363636,30,0,30,0,60


In [19]:
#calculate consistency
def calculate_consistency_bowler(df, bowler_id):
    # Filter data for the specific bowler
    bowler_df = df[df['bowler_id'] == bowler_id]
    bowler_df = bowler_df[bowler_df['wicket_count'] > 0]

    # Calculate strike rate: balls bowled per wicket taken
    bowler_df['bowler_strike_rate'] = bowler_df['balls_bowled'] / bowler_df['wicket_count']
    
    # Calculate the standard deviation of economy and strike rate
    std_eco = bowler_df['economy'].std()
    std_sr = bowler_df['bowler_strike_rate'].std()

    
    consistency = {
        'bowler_id': bowler_id,
        'std_economy': std_eco,
        'std_bowler_strike_rate': std_sr

    }
    
    return consistency

In [20]:
def calculate_consistency_for_all_bowlers(df,df_for_id):
    # Get a list of unique bowler IDs
    bowler_ids = df_for_id['bowler_id']
    
    results = []
    
    # Iterate over each bowler ID and calculate consistency
    for bowler_id in bowler_ids:
        consistency = calculate_consistency_bowler(df, bowler_id)
        results.append(consistency)
    
    # Convert the results list to a DataFrame
    consistency_df = pd.DataFrame(results)
    
    return consistency_df

consistency_df_bowl = calculate_consistency_for_all_bowlers(bowler_df,selected_bowler)
print(consistency_df_bowl)

     bowler_id  std_economy  std_bowler_strike_rate
0        34061     3.286222                4.904080
1        49496     1.922590                8.389912
2        55299     2.393895                6.964613
3        62432     4.116509                6.374948
4        65183     2.733365                4.867898
..         ...          ...                     ...
478    8833075     2.167391                5.558027
479    8845850     2.372230                5.647640
480    8907338     2.067828                6.969670
481    8986193     2.540279                4.927354
482    9455557     2.641248                9.044651

[483 rows x 3 columns]


In [21]:
# Calculate recency
def calculate_recency_and_performance_bowler(df, bowler_id, recent_matches=5):
    # Filter data for the specific bowler
    bowler_df = df[df['bowler_id'] == bowler_id]
    
    # Convert match dates to datetime format
    bowler_df['match_dt'] = pd.to_datetime(bowler_df['match_dt'])
    
    # Sort the matches by date in descending order to get the most recent matches
    bowler_df = bowler_df.sort_values(by='match_dt', ascending=False)
    
    # Select the most recent matches
    recent_bowler_df = bowler_df.head(recent_matches)
    
    # Calculate recency as the mean of the timestamps of the recent matches
    match_dates = recent_bowler_df['match_dt'].apply(lambda x: x.timestamp())
    recency_score = np.mean(match_dates)
    
    # Summarize performance metrics in recent matches
    total_wickets = recent_bowler_df['wicket_count'].sum()
    total_runs_conceded = recent_bowler_df['runs'].sum()
    matches_played = len(recent_bowler_df)
    average_runs_conceded = total_runs_conceded / total_wickets if total_wickets > 0 else 0
    
    recency = {
        'bowler_id': bowler_id,
        'recency_score': recency_score,
        'matches_played': matches_played,
        'total_wickets': total_wickets,
        'total_runs_conceded': total_runs_conceded,
        'average_runs_conceded': average_runs_conceded
    }
    
    return recency

In [22]:
def calculate_recency_for_all_bowlers(df,df_for_id):
    # Get a list of unique bowler IDs
    bowler_ids = df_for_id['bowler_id']
    
    results = []
    
    # Iterate over each bowler ID and calculate consistency
    for bowler_id in bowler_ids:
        recency = calculate_recency_and_performance_bowler(df, bowler_id)
        results.append(recency)
    
    # Convert the results list to a DataFrame
    recency_df = pd.DataFrame(results)
    
    return recency_df

recency_df_bowl = calculate_recency_for_all_bowlers(bowler_df,selected_bowler)
print(recency_df_bowl)

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_13712\553790713.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bowler_df['match_dt'] = pd.to_datetime(bowler_df['match_dt'])
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_13712\553790713.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bowler_df['match_dt'] = pd.to_datetime(bowler_df['match_dt'])
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_13712\553790713.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_13712\553790713.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bowler_df['match_dt'] = pd.to_datetime(bowler_df['match_dt'])
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_13712\553790713.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bowler_df['match_dt'] = pd.to_datetime(bowler_df['match_dt'])
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_13712\553790713.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_13712\553790713.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bowler_df['match_dt'] = pd.to_datetime(bowler_df['match_dt'])
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_13712\553790713.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bowler_df['match_dt'] = pd.to_datetime(bowler_df['match_dt'])
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_13712\553790713.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_13712\553790713.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bowler_df['match_dt'] = pd.to_datetime(bowler_df['match_dt'])
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_13712\553790713.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bowler_df['match_dt'] = pd.to_datetime(bowler_df['match_dt'])
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_13712\553790713.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_13712\553790713.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bowler_df['match_dt'] = pd.to_datetime(bowler_df['match_dt'])
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_13712\553790713.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bowler_df['match_dt'] = pd.to_datetime(bowler_df['match_dt'])
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_13712\553790713.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_13712\553790713.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bowler_df['match_dt'] = pd.to_datetime(bowler_df['match_dt'])
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_13712\553790713.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bowler_df['match_dt'] = pd.to_datetime(bowler_df['match_dt'])
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_13712\553790713.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_13712\553790713.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bowler_df['match_dt'] = pd.to_datetime(bowler_df['match_dt'])
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_13712\553790713.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bowler_df['match_dt'] = pd.to_datetime(bowler_df['match_dt'])
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_13712\553790713.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_13712\553790713.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bowler_df['match_dt'] = pd.to_datetime(bowler_df['match_dt'])
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_13712\553790713.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bowler_df['match_dt'] = pd.to_datetime(bowler_df['match_dt'])
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_13712\553790713.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try

     bowler_id  recency_score  matches_played  total_wickets  \
0        34061   1.669075e+09               5              2   
1        49496   1.680376e+09               5              8   
2        55299   1.692714e+09               5              3   
3        62432   1.697034e+09               5              2   
4        65183   1.625875e+09               5             11   
..         ...            ...             ...            ...   
478    8833075   1.688895e+09               5              9   
479    8845850   1.697138e+09               5              6   
480    8907338   1.696706e+09               5              5   
481    8986193   1.680134e+09               5              8   
482    9455557   1.692334e+09               5             10   

     total_runs_conceded  average_runs_conceded  
0                     73              36.500000  
1                    119              14.875000  
2                    150              50.000000  
3                     90       

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_13712\553790713.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bowler_df['match_dt'] = pd.to_datetime(bowler_df['match_dt'])
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_13712\553790713.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bowler_df['match_dt'] = pd.to_datetime(bowler_df['match_dt'])
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_13712\553790713.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try

In [23]:
c_and_r_bowler  =pd.merge(consistency_df_bowl, recency_df_bowl, on ='bowler_id')
c_and_r_bowler

,bowler_id,std_economy,std_bowler_strike_rate,recency_score,matches_played,total_wickets,total_runs_conceded,average_runs_conceded
0,34061,3.286222,4.904080,1.669075e+09,5,2,73,36.500000
1,49496,1.922590,8.389912,1.680376e+09,5,8,119,14.875000
2,55299,2.393895,6.964613,1.692714e+09,5,3,150,50.000000
3,62432,4.116509,6.374948,1.697034e+09,5,2,90,45.000000
4,65183,2.733365,4.867898,1.625875e+09,5,11,158,14.363636
...,...,...,...,...,...,...,...,...
478,8833075,2.167391,5.558027,1.688895e+09,5,9,77,8.555556
479,8845850,2.372230,5.647640,1.697138e+09,5,6,162,27.000000
480,8907338,2.067828,6.969670,1.696706e+09,5,5,148,29.600000
481,8986193,2.540279,4.927354,1.680134e+09,5,8,145,18.125000


In [24]:
final_bowler  =pd.merge(c_and_r_bowler, selected_bowler, on ='bowler_id')
final_bowler

,bowler_id,std_economy,std_bowler_strike_rate,recency_score,matches_played,total_wickets,total_runs_conceded,average_runs_conceded,cumulative_wickets,balls_bowled,runs_given,count_4w_per_inning,matches,economy,strike_rate_bowl,average_bowl,score_strike_bowl,score_economy,score_average_bowl,score_4w,total_score_bowl
0,34061,3.286222,4.904080,1.669075e+09,5,2,73,36.500000,27,635,965,0,46,9.490000,23.518519,35.740741,10,0,10,0,20
1,49496,1.922590,8.389912,1.680376e+09,5,8,119,14.875000,31,516,533,0,23,6.203043,16.645161,17.193548,20,30,30,0,80
2,55299,2.393895,6.964613,1.692714e+09,5,3,150,50.000000,46,709,978,0,35,8.097429,15.413043,21.260870,20,0,20,0,40
3,62432,4.116509,6.374948,1.697034e+09,5,2,90,45.000000,72,1178,1418,0,77,7.826494,16.361111,19.694444,20,0,30,0,50
4,65183,2.733365,4.867898,1.625875e+09,5,11,158,14.363636,16,189,280,1,9,9.090000,11.812500,17.500000,30,0,30,10,70
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
478,8833075,2.167391,5.558027,1.688895e+09,5,9,77,8.555556,21,225,239,0,11,6.221818,10.714286,11.380952,30,30,30,0,90
479,8845850,2.372230,5.647640,1.697138e+09,5,6,162,27.000000,32,638,866,0,31,8.374194,19.937500,27.062500,10,0,20,0,30
480,8907338,2.067828,6.969670,1.696706e+09,5,5,148,29.600000,47,837,1141,1,41,8.232439,17.808511,24.276596,20,0,20,10,50
481,8986193,2.540279,4.927354,1.680134e+09,5,8,145,18.125000,22,283,338,0,13,7.282308,12.863636,15.363636,30,0,30,0,60


In [25]:
final_bowler2 = final_bowler[final_bowler['recency_score']>1680000000]

In [26]:
# Adding normalised columns
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
final_bowler2['total_score_bowler_normalised'] = scaler.fit_transform(final_bowler2[['total_score_bowl']])
final_bowler2['std_bowler_economy_normalised'] = scaler.fit_transform(-final_bowler2[['std_economy']])
final_bowler2['recent_average_normalised'] = scaler.fit_transform(-final_bowler2[['average_runs_conceded']])
final_bowler2['sum'] = final_bowler2['total_score_bowler_normalised'] + final_bowler2['recent_average_normalised'] + final_bowler2['std_bowler_economy_normalised']
final_bowler2

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_13712\4186563684.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_bowler2['total_score_bowler_normalised'] = scaler.fit_transform(final_bowler2[['total_score_bowl']])
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_13712\4186563684.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_bowler2['std_bowler_economy_normalised'] = scaler.fit_transform(-final_bowler2[['std_economy']])
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_13712\4186563684.py:6: Settin

,bowler_id,std_economy,std_bowler_strike_rate,recency_score,matches_played,total_wickets,total_runs_conceded,average_runs_conceded,cumulative_wickets,balls_bowled,runs_given,count_4w_per_inning,matches,economy,strike_rate_bowl,average_bowl,score_strike_bowl,score_economy,score_average_bowl,score_4w,total_score_bowl,total_score_bowler_normalised,std_bowler_economy_normalised,recent_average_normalised,sum
1,49496,1.922590,8.389912,1.680376e+09,5,8,119,14.875000,31,516,533,0,23,6.203043,16.645161,17.193548,20,30,30,0,80,0.8,0.728479,0.907031,2.435510
2,55299,2.393895,6.964613,1.692714e+09,5,3,150,50.000000,46,709,978,0,35,8.097429,15.413043,21.260870,20,0,20,0,40,0.4,0.604216,0.687500,1.691716
3,62432,4.116509,6.374948,1.697034e+09,5,2,90,45.000000,72,1178,1418,0,77,7.826494,16.361111,19.694444,20,0,30,0,50,0.5,0.150037,0.718750,1.368787
5,74087,1.891041,6.192313,1.689863e+09,5,7,99,14.142857,30,609,790,0,42,8.110000,20.300000,26.333333,10,0,20,0,30,0.3,0.736797,0.911607,1.948404
6,74367,1.683813,6.472377,1.680947e+09,5,7,122,17.428571,15,260,344,0,14,8.254286,17.333333,22.933333,20,0,20,0,40,0.4,0.791434,0.891071,2.082506
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
478,8833075,2.167391,5.558027,1.688895e+09,5,9,77,8.555556,21,225,239,0,11,6.221818,10.714286,11.380952,30,30,30,0,90,0.9,0.663936,0.946528,2.510464
479,8845850,2.372230,5.647640,1.697138e+09,5,6,162,27.000000,32,638,866,0,31,8.374194,19.937500,27.062500,10,0,20,0,30,0.3,0.609929,0.831250,1.741179
480,8907338,2.067828,6.969670,1.696706e+09,5,5,148,29.600000,47,837,1141,1,41,8.232439,17.808511,24.276596,20,0,20,10,50,0.5,0.690186,0.815000,2.005186
481,8986193,2.540279,4.927354,1.680134e+09,5,8,145,18.125000,22,283,338,0,13,7.282308,12.863636,15.363636,30,0,30,0,60,0.6,0.565621,0.886719,2.052340


In [27]:
final_sorted_bowler = final_bowler2.sort_values(by=['sum'], ascending=[False])
final_sorted_bowler.head()

,bowler_id,std_economy,std_bowler_strike_rate,recency_score,matches_played,total_wickets,total_runs_conceded,average_runs_conceded,cumulative_wickets,balls_bowled,runs_given,count_4w_per_inning,matches,economy,strike_rate_bowl,average_bowl,score_strike_bowl,score_economy,score_average_bowl,score_4w,total_score_bowl,total_score_bowler_normalised,std_bowler_economy_normalised,recent_average_normalised,sum
463,8246440,1.534747,7.970315,1.698123e+09,5,9,97,10.777778,23,247,214,1,12,5.320000,10.739130,9.304348,30,30,30,10,100,1.0,0.830737,0.932639,2.763375
202,3633503,1.134196,7.277820,1.692818e+09,5,8,93,11.625000,11,174,163,0,10,5.841000,15.818182,14.818182,20,30,30,0,80,0.8,0.936345,0.927344,2.663688
464,8246573,1.832357,6.231401,1.689708e+09,5,10,125,12.500000,14,151,167,0,7,6.820000,10.785714,11.928571,30,30,30,0,90,0.9,0.752270,0.921875,2.574145
396,7276380,1.610741,5.771812,1.692040e+09,5,10,129,12.900000,46,553,746,2,28,8.255357,12.021739,16.217391,30,0,30,20,80,0.8,0.810700,0.919375,2.530075
478,8833075,2.167391,5.558027,1.688895e+09,5,9,77,8.555556,21,225,239,0,11,6.221818,10.714286,11.380952,30,30,30,0,90,0.9,0.663936,0.946528,2.510464


In [28]:
# comparing with near best 11 provided
bowl_best11_df =pd.merge(final_sorted_bowler, nearbest_11, left_on='bowler_id', right_on='Player_id', how='inner')
bowl_best11_df

,bowler_id,std_economy,std_bowler_strike_rate,recency_score,matches_played,total_wickets,total_runs_conceded,average_runs_conceded,cumulative_wickets,balls_bowled,runs_given,count_4w_per_inning,matches,economy,strike_rate_bowl,average_bowl,score_strike_bowl,score_economy,score_average_bowl,score_4w,total_score_bowl,total_score_bowler_normalised,std_bowler_economy_normalised,recent_average_normalised,sum,Player_id
0,8246573,1.832357,6.231401,1.689708e+09,5,10,125,12.500000,14,151,167,0,7,6.820000,10.785714,11.928571,30,30,30,0,90,0.9,0.752270,0.921875,2.574145,8246573
1,8986193,2.540279,4.927354,1.680134e+09,5,8,145,18.125000,22,283,338,0,13,7.282308,12.863636,15.363636,30,0,30,0,60,0.6,0.565621,0.886719,2.052340,8986193
2,4171460,1.200602,8.335030,1.690554e+09,5,5,140,28.000000,21,462,631,1,26,8.708462,22.000000,30.047619,10,0,10,10,30,0.3,0.918836,0.825000,2.043836,4171460
3,7758638,2.049473,6.604893,1.692438e+09,5,6,145,24.166667,40,951,1137,0,42,7.389762,23.775000,28.425000,10,0,20,0,30,0.3,0.695026,0.848958,1.843984,7758638
4,2275195,3.386464,6.131763,1.699678e+09,5,5,146,29.200000,36,882,1154,0,68,8.365588,24.500000,32.055556,0,0,10,0,10,0.1,0.342519,0.817500,1.260019,2275195


# WICKET KEEPER SELECTION

# not taking into account "is_bowler_keeper" because wk are judged based on their batsman score

In [29]:
def calculate_metrics(batsman_df):
    batsman_df['half_centuries'] = (batsman_df['runs'] >= 50).astype(int)
    batsman_df['centuries'] = (batsman_df['runs'] >= 100).astype(int)
    
    batsman_stats = batsman_df.groupby('batsman_id').agg({
        'runs': 'sum',
        'half_centuries': 'sum',
        'centuries': 'sum',
        'match id': 'nunique',
        'strike_rate': 'mean',
        'is_batsman_keeper': 'sum', 
    }).reset_index()
    batsman_stats.columns = ['batsman_id', 'cumulative_runs', 'half_centuries','centuries','matches','strike_rate','is_batsman_keeper']
    
    batsman_stats['average'] = batsman_stats['cumulative_runs'] / batsman_stats['matches']
    
    #scores assignment
    batsman_stats['score_strike'] = np.where(batsman_stats['strike_rate'] >= 150, 50,
                                      np.where((batsman_stats['strike_rate'] >= 100) & (batsman_stats['strike_rate'] < 150), 40,
                                               np.where((batsman_stats['strike_rate'] >= 80) & (batsman_stats['strike_rate'] < 100), 30,
                                                        0)))
    batsman_stats['score_half_centuries'] = np.where(batsman_stats['half_centuries'] >= 5, 20,
                                                     np.where((batsman_stats['half_centuries'] == 3) | (batsman_stats['half_centuries'] == 4), 10,
                                                              np.where((batsman_stats['half_centuries'] == 1) | (batsman_stats['half_centuries'] == 2), 5,
                                                                       0)))
    batsman_stats['score_centuries'] = np.where(batsman_stats['centuries'] >= 3, 30,
                                                     np.where((batsman_stats['centuries'] == 2), 20,
                                                              np.where((batsman_stats['centuries'] == 1), 10,
                                                                       0)))
    
    batsman_stats['score_average'] = np.where(batsman_stats['average'] >= 50, 30,
                                              np.where((batsman_stats['average'] >= 40) & (batsman_stats['average'] < 50), 20,
                                                       np.where((batsman_stats['average'] >= 30) & (batsman_stats['average'] < 40), 10,
                                                                5)))
    
    
    batsman_stats['total_score'] = batsman_stats['score_strike'] + batsman_stats['score_half_centuries'] + batsman_stats['score_centuries'] + batsman_stats['score_average']
    wk_stats = batsman_stats[(batsman_stats['is_batsman_keeper'] >= 2 ) & (batsman_stats['matches'] >= 2)]
    return wk_stats

wk_stats = calculate_metrics(batsman_df)
wk_stats

,batsman_id,cumulative_runs,half_centuries,centuries,matches,strike_rate,is_batsman_keeper,average,score_strike,score_half_centuries,score_centuries,score_average,total_score
18,87163,24,0,0,2,81.110000,2,12.000000,30,0,0,5,35
27,150093,506,2,0,39,115.135128,39,12.974359,40,5,0,5,50
46,196580,338,1,0,27,126.612593,27,12.518519,40,5,0,5,50
50,210328,980,2,0,59,137.973559,48,16.610169,40,5,0,5,50
51,211658,140,1,0,4,84.737500,4,35.000000,30,5,0,10,45
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2003,9103996,17,0,0,3,44.443333,3,5.666667,0,0,0,5,5
2013,9227455,17,0,0,2,73.160000,2,8.500000,0,0,0,5,5
2015,9245319,14,0,0,3,103.333333,2,4.666667,40,0,0,5,45
2026,9346861,45,0,0,3,97.060000,3,15.000000,30,0,0,5,35


In [30]:
# calculate consistency
consistency_df_wk = calculate_consistency_for_all_batsman(batsman_df, wk_stats)
print(consistency_df_wk)

     batsman_id  std_strike_rate
0         87163        58.138320
1        150093        98.718005
2        196580        95.421785
3        210328        92.617033
4        211658        29.569459
..          ...              ...
191     9103996        34.691765
192     9227455        20.279822
193     9245319        55.075705
194     9346861        45.661042
195     9822259        20.202041

[196 rows x 2 columns]


In [31]:
# calculate recency
recency_df_wk = calculate_recency_for_all_batsman(batsman_df,wk_stats)
print(recency_df_wk)

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_13712\3194042368.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batsman_df['match_dt'] = pd.to_datetime(batsman_df['match_dt'])
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_13712\3194042368.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batsman_df['match_dt'] = pd.to_datetime(batsman_df['match_dt'])
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_13712\3194042368.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFra

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_13712\3194042368.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batsman_df['match_dt'] = pd.to_datetime(batsman_df['match_dt'])
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_13712\3194042368.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batsman_df['match_dt'] = pd.to_datetime(batsman_df['match_dt'])
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_13712\3194042368.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFra

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_13712\3194042368.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batsman_df['match_dt'] = pd.to_datetime(batsman_df['match_dt'])
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_13712\3194042368.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batsman_df['match_dt'] = pd.to_datetime(batsman_df['match_dt'])
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_13712\3194042368.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFra

     batsman_id  recency_score  matches_played  total_runs  total_balls_faced  \
0         87163   1.626480e+09               2          24                 23   
1        150093   1.687150e+09               5          56                 41   
2        196580   1.684022e+09               5          10                 13   
3        210328   1.682709e+09               5          61                 49   
4        211658   1.636200e+09               4         140                145   
..          ...            ...             ...         ...                ...   
191     9103996   1.672358e+09               3          17                 27   
192     9227455   1.673741e+09               2          17                 25   
193     9245319   1.697962e+09               3          14                 13   
194     9346861   1.676045e+09               3          45                 45   
195     9822259   1.697933e+09               2          52                 23   

      recent_sr  
0    104.

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_13712\3194042368.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batsman_df['match_dt'] = pd.to_datetime(batsman_df['match_dt'])
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_13712\3194042368.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batsman_df['match_dt'] = pd.to_datetime(batsman_df['match_dt'])
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_13712\3194042368.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFra

In [32]:
c_and_r_wk  =pd.merge(consistency_df_wk, recency_df_wk, on ='batsman_id')
c_and_r_wk

,batsman_id,std_strike_rate,recency_score,matches_played,total_runs,total_balls_faced,recent_sr
0,87163,58.138320,1.626480e+09,2,24,23,104.347826
1,150093,98.718005,1.687150e+09,5,56,41,136.585366
2,196580,95.421785,1.684022e+09,5,10,13,76.923077
3,210328,92.617033,1.682709e+09,5,61,49,124.489796
4,211658,29.569459,1.636200e+09,4,140,145,96.551724
...,...,...,...,...,...,...,...
191,9103996,34.691765,1.672358e+09,3,17,27,62.962963
192,9227455,20.279822,1.673741e+09,2,17,25,68.000000
193,9245319,55.075705,1.697962e+09,3,14,13,107.692308
194,9346861,45.661042,1.676045e+09,3,45,45,100.000000


In [33]:
final_wk  =pd.merge(c_and_r_wk, wk_stats, on ='batsman_id')
final_wk

,batsman_id,std_strike_rate,recency_score,matches_played,total_runs,total_balls_faced,recent_sr,cumulative_runs,half_centuries,centuries,matches,strike_rate,is_batsman_keeper,average,score_strike,score_half_centuries,score_centuries,score_average,total_score
0,87163,58.138320,1.626480e+09,2,24,23,104.347826,24,0,0,2,81.110000,2,12.000000,30,0,0,5,35
1,150093,98.718005,1.687150e+09,5,56,41,136.585366,506,2,0,39,115.135128,39,12.974359,40,5,0,5,50
2,196580,95.421785,1.684022e+09,5,10,13,76.923077,338,1,0,27,126.612593,27,12.518519,40,5,0,5,50
3,210328,92.617033,1.682709e+09,5,61,49,124.489796,980,2,0,59,137.973559,48,16.610169,40,5,0,5,50
4,211658,29.569459,1.636200e+09,4,140,145,96.551724,140,1,0,4,84.737500,4,35.000000,30,5,0,10,45
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
191,9103996,34.691765,1.672358e+09,3,17,27,62.962963,17,0,0,3,44.443333,3,5.666667,0,0,0,5,5
192,9227455,20.279822,1.673741e+09,2,17,25,68.000000,17,0,0,2,73.160000,2,8.500000,0,0,0,5,5
193,9245319,55.075705,1.697962e+09,3,14,13,107.692308,14,0,0,3,103.333333,2,4.666667,40,0,0,5,45
194,9346861,45.661042,1.676045e+09,3,45,45,100.000000,45,0,0,3,97.060000,3,15.000000,30,0,0,5,35


In [34]:
low = final_wk['recency_score'].max()
high = final_wk['recency_score'].min()
mid = (low+high)/2
final_wk2 = final_wk[final_wk['recency_score']>mid]

In [35]:
# Adding normalised columns
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
final_wk2['total_score_normalised'] = scaler.fit_transform(final_wk2[['total_score']])
final_wk2['std_sr_normalised'] = scaler.fit_transform(-final_wk2[['std_strike_rate']])
final_wk2['recent_sr_normalised'] = scaler.fit_transform(final_wk2[['recent_sr']])
final_wk2['recent_matches_played'] = scaler.fit_transform(final_wk2[['matches_played']])
final_wk2['sum'] = final_wk2['std_sr_normalised'] + final_wk2['recent_sr_normalised'] + final_wk2['total_score_normalised'] + final_wk2['recent_matches_played']
final_wk2

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_13712\3885437773.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_wk2['total_score_normalised'] = scaler.fit_transform(final_wk2[['total_score']])
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_13712\3885437773.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_wk2['std_sr_normalised'] = scaler.fit_transform(-final_wk2[['std_strike_rate']])
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_13712\3885437773.py:6: SettingWithCopyWarning: 
A value is trying

,batsman_id,std_strike_rate,recency_score,matches_played,total_runs,total_balls_faced,recent_sr,cumulative_runs,half_centuries,centuries,matches,strike_rate,is_batsman_keeper,average,score_strike,score_half_centuries,score_centuries,score_average,total_score,total_score_normalised,std_sr_normalised,recent_sr_normalised,recent_matches_played,sum
1,150093,98.718005,1.687150e+09,5,56,41,136.585366,506,2,0,39,115.135128,39,12.974359,40,5,0,5,50,0.473684,0.000000,0.511534,1.000000,1.985218
2,196580,95.421785,1.684022e+09,5,10,13,76.923077,338,1,0,27,126.612593,27,12.518519,40,5,0,5,50,0.473684,0.037374,0.185919,1.000000,1.696977
3,210328,92.617033,1.682709e+09,5,61,49,124.489796,980,2,0,59,137.973559,48,16.610169,40,5,0,5,50,0.473684,0.069175,0.445521,1.000000,1.988380
8,363047,61.083692,1.693475e+09,5,106,77,137.662338,1436,8,2,57,112.297368,27,25.192982,40,20,20,5,85,0.842105,0.426711,0.517411,1.000000,2.786228
10,392216,57.542840,1.676056e+09,5,203,145,140.000000,688,4,0,33,94.050303,20,20.848485,30,10,0,5,45,0.421053,0.466858,0.530169,1.000000,2.418081
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
191,9103996,34.691765,1.672358e+09,3,17,27,62.962963,17,0,0,3,44.443333,3,5.666667,0,0,0,5,5,0.000000,0.725952,0.109730,0.333333,1.169015
192,9227455,20.279822,1.673741e+09,2,17,25,68.000000,17,0,0,2,73.160000,2,8.500000,0,0,0,5,5,0.000000,0.889360,0.137220,0.000000,1.026580
193,9245319,55.075705,1.697962e+09,3,14,13,107.692308,14,0,0,3,103.333333,2,4.666667,40,0,0,5,45,0.421053,0.494832,0.353846,0.333333,1.603064
194,9346861,45.661042,1.676045e+09,3,45,45,100.000000,45,0,0,3,97.060000,3,15.000000,30,0,0,5,35,0.315789,0.601578,0.311864,0.333333,1.562566


In [36]:
final_sorted_wk = final_wk2.sort_values(by=['sum'], ascending=[False])
final_sorted_wk.head()

,batsman_id,std_strike_rate,recency_score,matches_played,total_runs,total_balls_faced,recent_sr,cumulative_runs,half_centuries,centuries,matches,strike_rate,is_batsman_keeper,average,score_strike,score_half_centuries,score_centuries,score_average,total_score,total_score_normalised,std_sr_normalised,recent_sr_normalised,recent_matches_played,sum
31,2263736,43.395586,1.680670e+09,5,179,134,133.582090,4031,40,2,92,115.673261,84,43.815217,40,20,20,20,100,1.000000,0.627265,0.495143,1.0,3.122408
29,2162782,58.786308,1.697103e+09,5,233,149,156.375839,3498,32,6,97,123.307835,61,36.061856,40,20,30,10,100,1.000000,0.452760,0.619543,1.0,3.072302
108,4685568,60.998638,1.696861e+09,5,249,122,204.098361,2525,15,1,105,132.002952,63,24.047619,40,20,10,5,75,0.736842,0.427675,0.879994,1.0,3.044512
100,4230127,68.570623,1.697708e+09,5,220,120,183.333333,2176,11,1,101,120.201287,58,21.544554,40,20,10,5,75,0.736842,0.341822,0.766667,1.0,2.845330
92,4069666,46.989264,1.698227e+09,5,259,165,156.969697,873,4,1,32,118.593437,12,27.281250,40,10,10,5,65,0.631579,0.586519,0.622784,1.0,2.840881


In [37]:
# comparing with near best 11 provided
wk_best11_df =pd.merge(final_sorted_wk, nearbest_11, left_on='batsman_id', right_on='Player_id', how='inner')
wk_best11_df

,batsman_id,std_strike_rate,recency_score,matches_played,total_runs,total_balls_faced,recent_sr,cumulative_runs,half_centuries,centuries,matches,strike_rate,is_batsman_keeper,average,score_strike,score_half_centuries,score_centuries,score_average,total_score,total_score_normalised,std_sr_normalised,recent_sr_normalised,recent_matches_played,sum,Player_id
0,8250115,44.223173,1.699799e+09,5,160,152,105.263158,700,6,0,21,100.807619,13,33.333333,40,20,0,10,70,0.684211,0.617881,0.340589,1.0,2.642681,8250115


# ALL ROUNDER SELECTION

In [38]:
selected_allround_from_bat = batsman_stats[batsman_stats['cumulative_runs'] > 10]
selected_allround_from_bowl = bowler_stats[bowler_stats['cumulative_wickets'] >= 1]
selected_allround_from_bowl2 = selected_allround_from_bowl[selected_allround_from_bowl['matches'] >= 2]
  

allround_df =pd.merge(selected_allround_from_bat, selected_allround_from_bowl2, left_on='batsman_id', right_on='bowler_id', how='inner')
allround_df['total_score_allround'] = allround_df['total_score'] + allround_df['total_score_bowl']
allround_df

,batsman_id,cumulative_runs,half_centuries,centuries,matches_x,strike_rate,average,score_strike,score_half_centuries,score_centuries,score_average,total_score,bowler_id,cumulative_wickets,balls_bowled,runs_given,count_4w_per_inning,matches_y,economy,strike_rate_bowl,average_bowl,score_strike_bowl,score_economy,score_average_bowl,score_4w,total_score_bowl,total_score_allround
0,34061,729,2,0,52,119.160577,14.019231,40,5,0,5,50,34061,27,635,965,0,46,9.490000,23.518519,35.740741,10,0,10,0,20,70
1,41740,976,6,0,41,116.654878,23.804878,40,20,0,5,65,41740,5,90,76,0,10,5.900000,18.000000,15.200000,20,30,30,0,80,145
2,49496,23,0,0,5,106.192000,4.600000,40,0,0,5,45,49496,31,516,533,0,23,6.203043,16.645161,17.193548,20,30,30,0,80,125
3,55299,20,0,0,6,79.166667,3.333333,0,0,0,5,5,55299,46,709,978,0,35,8.097429,15.413043,21.260870,20,0,20,0,40,45
4,62432,1911,7,0,87,121.130345,21.965517,40,20,0,5,65,62432,72,1178,1418,0,77,7.826494,16.361111,19.694444,20,0,30,0,50,115
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
789,9266102,27,0,0,2,97.180000,13.500000,30,0,0,5,35,9266102,2,48,44,0,3,5.333333,24.000000,22.000000,10,30,20,0,60,95
790,9307892,11,0,0,3,66.346667,3.666667,0,0,0,5,5,9307892,6,114,163,0,7,9.392857,19.000000,27.166667,20,0,20,0,40,45
791,9329494,13,0,0,1,92.860000,13.000000,30,0,0,5,35,9329494,2,24,26,0,2,9.335000,12.000000,13.000000,30,0,30,0,60,95
792,9346854,63,0,0,3,144.973333,21.000000,40,0,0,5,45,9346854,2,52,77,0,3,8.416667,26.000000,38.500000,0,0,10,0,10,55


In [39]:
# calculate consistency
consistency_df_allrounder = calculate_consistency_for_all_batsman(batsman_df, allround_df)
print(consistency_df_allrounder)

     batsman_id  std_strike_rate
0         34061        87.348364
1         41740        53.823271
2         49496        76.849510
3         55299       101.070383
4         62432        62.318474
..          ...              ...
789     9266102        14.863385
790     9307892        28.735453
791     9329494              NaN
792     9346854        26.898605
793     9413816       169.705627

[794 rows x 2 columns]


In [40]:
consistency_df_allrounder2 = calculate_consistency_for_all_bowlers(bowler_df,allround_df)
print(consistency_df_allrounder2)

     bowler_id  std_economy  std_bowler_strike_rate
0        34061     3.286222                4.904080
1        41740     1.527525                1.154701
2        49496     1.922590                8.389912
3        55299     2.393895                6.964613
4        62432     4.116509                6.374948
..         ...          ...                     ...
789    9266102     0.473762                4.242641
790    9307892     4.740671                8.246211
791    9329494          NaN                     NaN
792    9346854     3.358757                8.485281
793    9413816          NaN                     NaN

[794 rows x 3 columns]


In [41]:
# calculate rcency
recency_df_allrounder = calculate_recency_for_all_batsman(batsman_df, allround_df)
print(recency_df_allrounder)

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_13712\3194042368.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batsman_df['match_dt'] = pd.to_datetime(batsman_df['match_dt'])
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_13712\3194042368.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batsman_df['match_dt'] = pd.to_datetime(batsman_df['match_dt'])
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_13712\3194042368.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFra

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_13712\3194042368.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batsman_df['match_dt'] = pd.to_datetime(batsman_df['match_dt'])
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_13712\3194042368.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batsman_df['match_dt'] = pd.to_datetime(batsman_df['match_dt'])
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_13712\3194042368.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFra

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_13712\3194042368.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batsman_df['match_dt'] = pd.to_datetime(batsman_df['match_dt'])
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_13712\3194042368.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batsman_df['match_dt'] = pd.to_datetime(batsman_df['match_dt'])
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_13712\3194042368.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFra

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_13712\3194042368.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batsman_df['match_dt'] = pd.to_datetime(batsman_df['match_dt'])
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_13712\3194042368.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batsman_df['match_dt'] = pd.to_datetime(batsman_df['match_dt'])
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_13712\3194042368.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFra

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_13712\3194042368.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batsman_df['match_dt'] = pd.to_datetime(batsman_df['match_dt'])
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_13712\3194042368.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batsman_df['match_dt'] = pd.to_datetime(batsman_df['match_dt'])
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_13712\3194042368.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFra

...

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_13712\3194042368.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batsman_df['match_dt'] = pd.to_datetime(batsman_df['match_dt'])
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_13712\3194042368.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batsman_df['match_dt'] = pd.to_datetime(batsman_df['match_dt'])
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_13712\3194042368.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFra

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_13712\3194042368.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batsman_df['match_dt'] = pd.to_datetime(batsman_df['match_dt'])
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_13712\3194042368.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batsman_df['match_dt'] = pd.to_datetime(batsman_df['match_dt'])
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_13712\3194042368.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFra

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_13712\3194042368.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batsman_df['match_dt'] = pd.to_datetime(batsman_df['match_dt'])
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_13712\3194042368.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batsman_df['match_dt'] = pd.to_datetime(batsman_df['match_dt'])
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_13712\3194042368.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFra

     batsman_id  recency_score  matches_played  total_runs  total_balls_faced  \
0         34061   1.673914e+09               5          38                 32   
1         41740   1.680376e+09               5         116                102   
2         49496   1.635967e+09               5          23                 24   
3         55299   1.661731e+09               5          20                 20   
4         62432   1.695082e+09               5         115                 80   
..          ...            ...             ...         ...                ...   
789     9266102   1.663675e+09               2          27                 28   
790     9307892   1.683907e+09               3          11                 15   
791     9329494   1.696550e+09               1          13                 14   
792     9346854   1.676045e+09               3          63                 44   
793     9413816   1.696982e+09               2          12                  6   

      recent_sr  
0    118.

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_13712\3194042368.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batsman_df['match_dt'] = pd.to_datetime(batsman_df['match_dt'])
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_13712\3194042368.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batsman_df['match_dt'] = pd.to_datetime(batsman_df['match_dt'])
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_13712\3194042368.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFra

In [42]:
recency_df_allrounder2 = calculate_recency_for_all_bowlers(bowler_df,allround_df)
print(recency_df_allrounder2)

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_13712\553790713.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bowler_df['match_dt'] = pd.to_datetime(bowler_df['match_dt'])
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_13712\553790713.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bowler_df['match_dt'] = pd.to_datetime(bowler_df['match_dt'])
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_13712\553790713.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_13712\553790713.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bowler_df['match_dt'] = pd.to_datetime(bowler_df['match_dt'])
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_13712\553790713.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bowler_df['match_dt'] = pd.to_datetime(bowler_df['match_dt'])
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_13712\553790713.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_13712\553790713.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bowler_df['match_dt'] = pd.to_datetime(bowler_df['match_dt'])
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_13712\553790713.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bowler_df['match_dt'] = pd.to_datetime(bowler_df['match_dt'])
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_13712\553790713.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_13712\553790713.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bowler_df['match_dt'] = pd.to_datetime(bowler_df['match_dt'])
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_13712\553790713.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bowler_df['match_dt'] = pd.to_datetime(bowler_df['match_dt'])
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_13712\553790713.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_13712\553790713.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bowler_df['match_dt'] = pd.to_datetime(bowler_df['match_dt'])
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_13712\553790713.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bowler_df['match_dt'] = pd.to_datetime(bowler_df['match_dt'])
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_13712\553790713.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try

...

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_13712\553790713.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bowler_df['match_dt'] = pd.to_datetime(bowler_df['match_dt'])
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_13712\553790713.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bowler_df['match_dt'] = pd.to_datetime(bowler_df['match_dt'])
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_13712\553790713.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_13712\553790713.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bowler_df['match_dt'] = pd.to_datetime(bowler_df['match_dt'])
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_13712\553790713.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bowler_df['match_dt'] = pd.to_datetime(bowler_df['match_dt'])
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_13712\553790713.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_13712\553790713.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bowler_df['match_dt'] = pd.to_datetime(bowler_df['match_dt'])
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_13712\553790713.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bowler_df['match_dt'] = pd.to_datetime(bowler_df['match_dt'])
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_13712\553790713.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try

     bowler_id  recency_score  matches_played  total_wickets  \
0        34061   1.669075e+09               5              2   
1        41740   1.651847e+09               5              1   
2        49496   1.680376e+09               5              8   
3        55299   1.692714e+09               5              3   
4        62432   1.697034e+09               5              2   
..         ...            ...             ...            ...   
789    9266102   1.673654e+09               3              2   
790    9307892   1.685146e+09               5              5   
791    9329494   1.696594e+09               2              2   
792    9346854   1.676045e+09               3              2   
793    9413816   1.696982e+09               2              2   

     total_runs_conceded  average_runs_conceded  
0                     73                 36.500  
1                     35                 35.000  
2                    119                 14.875  
3                    150       

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_13712\553790713.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bowler_df['match_dt'] = pd.to_datetime(bowler_df['match_dt'])
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_13712\553790713.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bowler_df['match_dt'] = pd.to_datetime(bowler_df['match_dt'])
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_13712\553790713.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try

In [43]:
c_allrounder  =pd.merge(consistency_df_allrounder, consistency_df_allrounder2, left_on ='batsman_id', right_on='bowler_id')
c_allrounder

,batsman_id,std_strike_rate,bowler_id,std_economy,std_bowler_strike_rate
0,34061,87.348364,34061,3.286222,4.904080
1,41740,53.823271,41740,1.527525,1.154701
2,49496,76.849510,49496,1.922590,8.389912
3,55299,101.070383,55299,2.393895,6.964613
4,62432,62.318474,62432,4.116509,6.374948
...,...,...,...,...,...
789,9266102,14.863385,9266102,0.473762,4.242641
790,9307892,28.735453,9307892,4.740671,8.246211
791,9329494,NaN,9329494,NaN,NaN
792,9346854,26.898605,9346854,3.358757,8.485281


In [44]:
r_allrounder  =pd.merge(recency_df_allrounder, recency_df_allrounder2, left_on ='batsman_id', right_on='bowler_id')
r_allrounder

,batsman_id,recency_score_x,matches_played_x,total_runs,total_balls_faced,recent_sr,bowler_id,recency_score_y,matches_played_y,total_wickets,total_runs_conceded,average_runs_conceded
0,34061,1.673914e+09,5,38,32,118.750000,34061,1.669075e+09,5,2,73,36.500
1,41740,1.680376e+09,5,116,102,113.725490,41740,1.651847e+09,5,1,35,35.000
2,49496,1.635967e+09,5,23,24,95.833333,49496,1.680376e+09,5,8,119,14.875
3,55299,1.661731e+09,5,20,20,100.000000,55299,1.692714e+09,5,3,150,50.000
4,62432,1.695082e+09,5,115,80,143.750000,62432,1.697034e+09,5,2,90,45.000
...,...,...,...,...,...,...,...,...,...,...,...,...
789,9266102,1.663675e+09,2,27,28,96.428571,9266102,1.673654e+09,3,2,44,22.000
790,9307892,1.683907e+09,3,11,15,73.333333,9307892,1.685146e+09,5,5,113,22.600
791,9329494,1.696550e+09,1,13,14,92.857143,9329494,1.696594e+09,2,2,26,13.000
792,9346854,1.676045e+09,3,63,44,143.181818,9346854,1.676045e+09,3,2,77,38.500


In [45]:
c_and_r_allrounder  =pd.merge(c_allrounder, r_allrounder, on ='batsman_id')
c_and_r_allrounder

,batsman_id,std_strike_rate,bowler_id_x,std_economy,std_bowler_strike_rate,recency_score_x,matches_played_x,total_runs,total_balls_faced,recent_sr,bowler_id_y,recency_score_y,matches_played_y,total_wickets,total_runs_conceded,average_runs_conceded
0,34061,87.348364,34061,3.286222,4.904080,1.673914e+09,5,38,32,118.750000,34061,1.669075e+09,5,2,73,36.500
1,41740,53.823271,41740,1.527525,1.154701,1.680376e+09,5,116,102,113.725490,41740,1.651847e+09,5,1,35,35.000
2,49496,76.849510,49496,1.922590,8.389912,1.635967e+09,5,23,24,95.833333,49496,1.680376e+09,5,8,119,14.875
3,55299,101.070383,55299,2.393895,6.964613,1.661731e+09,5,20,20,100.000000,55299,1.692714e+09,5,3,150,50.000
4,62432,62.318474,62432,4.116509,6.374948,1.695082e+09,5,115,80,143.750000,62432,1.697034e+09,5,2,90,45.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
789,9266102,14.863385,9266102,0.473762,4.242641,1.663675e+09,2,27,28,96.428571,9266102,1.673654e+09,3,2,44,22.000
790,9307892,28.735453,9307892,4.740671,8.246211,1.683907e+09,3,11,15,73.333333,9307892,1.685146e+09,5,5,113,22.600
791,9329494,NaN,9329494,NaN,NaN,1.696550e+09,1,13,14,92.857143,9329494,1.696594e+09,2,2,26,13.000
792,9346854,26.898605,9346854,3.358757,8.485281,1.676045e+09,3,63,44,143.181818,9346854,1.676045e+09,3,2,77,38.500


In [46]:
final_allrounder  =pd.merge(c_and_r_allrounder, allround_df, on ='batsman_id')
final_allrounder

,batsman_id,std_strike_rate,bowler_id_x,std_economy,std_bowler_strike_rate,recency_score_x,matches_played_x,total_runs,total_balls_faced,recent_sr,bowler_id_y,recency_score_y,matches_played_y,total_wickets,total_runs_conceded,average_runs_conceded,cumulative_runs,half_centuries,centuries,matches_x,strike_rate,average,score_strike,score_half_centuries,score_centuries,score_average,total_score,bowler_id,cumulative_wickets,balls_bowled,runs_given,count_4w_per_inning,matches_y,economy,strike_rate_bowl,average_bowl,score_strike_bowl,score_economy,score_average_bowl,score_4w,total_score_bowl,total_score_allround
0,34061,87.348364,34061,3.286222,4.904080,1.673914e+09,5,38,32,118.750000,34061,1.669075e+09,5,2,73,36.500,729,2,0,52,119.160577,14.019231,40,5,0,5,50,34061,27,635,965,0,46,9.490000,23.518519,35.740741,10,0,10,0,20,70
1,41740,53.823271,41740,1.527525,1.154701,1.680376e+09,5,116,102,113.725490,41740,1.651847e+09,5,1,35,35.000,976,6,0,41,116.654878,23.804878,40,20,0,5,65,41740,5,90,76,0,10,5.900000,18.000000,15.200000,20,30,30,0,80,145
2,49496,76.849510,49496,1.922590,8.389912,1.635967e+09,5,23,24,95.833333,49496,1.680376e+09,5,8,119,14.875,23,0,0,5,106.192000,4.600000,40,0,0,5,45,49496,31,516,533,0,23,6.203043,16.645161,17.193548,20,30,30,0,80,125
3,55299,101.070383,55299,2.393895,6.964613,1.661731e+09,5,20,20,100.000000,55299,1.692714e+09,5,3,150,50.000,20,0,0,6,79.166667,3.333333,0,0,0,5,5,55299,46,709,978,0,35,8.097429,15.413043,21.260870,20,0,20,0,40,45
4,62432,62.318474,62432,4.116509,6.374948,1.695082e+09,5,115,80,143.750000,62432,1.697034e+09,5,2,90,45.000,1911,7,0,87,121.130345,21.965517,40,20,0,5,65,62432,72,1178,1418,0,77,7.826494,16.361111,19.694444,20,0,30,0,50,115
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
789,9266102,14.863385,9266102,0.473762,4.242641,1.663675e+09,2,27,28,96.428571,9266102,1.673654e+09,3,2,44,22.000,27,0,0,2,97.180000,13.500000,30,0,0,5,35,9266102,2,48,44,0,3,5.333333,24.000000,22.000000,10,30,20,0,60,95
790,9307892,28.735453,9307892,4.740671,8.246211,1.683907e+09,3,11,15,73.333333,9307892,1.685146e+09,5,5,113,22.600,11,0,0,3,66.346667,3.666667,0,0,0,5,5,9307892,6,114,163,0,7,9.392857,19.000000,27.166667,20,0,20,0,40,45
791,9329494,NaN,9329494,NaN,NaN,1.696550e+09,1,13,14,92.857143,9329494,1.696594e+09,2,2,26,13.000,13,0,0,1,92.860000,13.000000,30,0,0,5,35,9329494,2,24,26,0,2,9.335000,12.000000,13.000000,30,0,30,0,60,95
792,9346854,26.898605,9346854,3.358757,8.485281,1.676045e+09,3,63,44,143.181818,9346854,1.676045e+09,3,2,77,38.500,63,0,0,3,144.973333,21.000000,40,0,0,5,45,9346854,2,52,77,0,3,8.416667,26.000000,38.500000,0,0,10,0,10,55


In [47]:
low = final_allrounder['recency_score_x'].max()
high = final_allrounder['recency_score_x'].min()
mid = (low+high)/2
final_allrounder2 = final_allrounder[final_allrounder['recency_score_x']>mid]

In [48]:
low = final_allrounder2['recency_score_y'].max()
high = final_allrounder2['recency_score_y'].min()
mid = (low+high)/2
final_allrounder3 = final_allrounder2[final_allrounder2['recency_score_y']>mid]

In [49]:
# Adding normalised columns
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
final_allrounder3['total_score_normalised'] = scaler.fit_transform(final_allrounder3[['total_score_allround']])
final_allrounder3['std_sr_normalised'] = scaler.fit_transform(-final_allrounder3[['std_strike_rate']])
final_allrounder3['recent_sr_normalised'] = scaler.fit_transform(final_allrounder3[['recent_sr']])
final_allrounder3['recent_matches_batsman'] = scaler.fit_transform(final_allrounder3[['matches_played_x']])
final_allrounder3['recent_matches_bowler'] = scaler.fit_transform(final_allrounder3[['matches_played_y']])
final_allrounder3['std_bowler_economy_normalised'] = scaler.fit_transform(-final_allrounder3[['std_economy']])
final_allrounder3['recent_average_normalised'] = scaler.fit_transform(-final_allrounder3[['average_runs_conceded']])
final_allrounder3['sum'] = final_allrounder3['std_sr_normalised'] + final_allrounder3['recent_sr_normalised'] + final_allrounder3['total_score_normalised']+final_allrounder3['std_bowler_economy_normalised']+final_allrounder3['recent_average_normalised'] + final_allrounder3['recent_matches_batsman'] + final_allrounder3['recent_matches_bowler']
final_allrounder3

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_13712\93044396.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_allrounder3['total_score_normalised'] = scaler.fit_transform(final_allrounder3[['total_score_allround']])
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_13712\93044396.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_allrounder3['std_sr_normalised'] = scaler.fit_transform(-final_allrounder3[['std_strike_rate']])
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_13712\93044396.py:6: Setting

,batsman_id,std_strike_rate,bowler_id_x,std_economy,std_bowler_strike_rate,recency_score_x,matches_played_x,total_runs,total_balls_faced,recent_sr,bowler_id_y,recency_score_y,matches_played_y,total_wickets,total_runs_conceded,average_runs_conceded,cumulative_runs,half_centuries,centuries,matches_x,strike_rate,average,score_strike,score_half_centuries,score_centuries,score_average,total_score,bowler_id,cumulative_wickets,balls_bowled,runs_given,count_4w_per_inning,matches_y,economy,strike_rate_bowl,average_bowl,score_strike_bowl,score_economy,score_average_bowl,score_4w,total_score_bowl,total_score_allround,total_score_normalised,std_sr_normalised,recent_sr_normalised,recent_matches_batsman,recent_matches_bowler,std_bowler_economy_normalised,recent_average_normalised,sum
0,34061,87.348364,34061,3.286222,4.904080,1.673914e+09,5,38,32,118.750000,34061,1.669075e+09,5,2,73,36.500000,729,2,0,52,119.160577,14.019231,40,5,0,5,50,34061,27,635,965,0,46,9.490000,23.518519,35.740741,10,0,10,0,20,70,0.433333,0.661372,0.398148,1.00,1.000000,0.504189,0.771875,4.768917
3,55299,101.070383,55299,2.393895,6.964613,1.661731e+09,5,20,20,100.000000,55299,1.692714e+09,5,3,150,50.000000,20,0,0,6,79.166667,3.333333,0,0,0,5,5,55299,46,709,978,0,35,8.097429,15.413043,21.260870,20,0,20,0,40,45,0.266667,0.605407,0.325926,1.00,1.000000,0.638820,0.687500,4.524319
4,62432,62.318474,62432,4.116509,6.374948,1.695082e+09,5,115,80,143.750000,62432,1.697034e+09,5,2,90,45.000000,1911,7,0,87,121.130345,21.965517,40,20,0,5,65,62432,72,1178,1418,0,77,7.826494,16.361111,19.694444,20,0,30,0,50,115,0.733333,0.763456,0.494444,1.00,1.000000,0.378919,0.718750,5.088903
6,74087,56.746410,74087,1.891041,6.192313,1.689068e+09,5,79,74,106.756757,74087,1.689863e+09,5,7,99,14.142857,1338,10,0,65,103.259385,20.584615,40,20,0,5,65,74087,30,609,790,0,42,8.110000,20.300000,26.333333,10,0,20,0,30,95,0.600000,0.786181,0.351952,1.00,1.000000,0.714688,0.911607,5.364428
9,87191,58.744637,87191,1.879594,6.236096,1.687185e+09,5,190,106,179.245283,87191,1.686493e+09,5,1,64,64.000000,1127,4,0,52,98.856538,21.673077,30,10,0,5,45,87191,13,302,398,0,27,9.134074,23.230769,30.615385,10,0,10,0,20,65,0.400000,0.778032,0.631167,1.00,1.000000,0.716415,0.600000,5.125614
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
789,9266102,14.863385,9266102,0.473762,4.242641,1.663675e+09,2,27,28,96.428571,9266102,1.673654e+09,3,2,44,22.000000,27,0,0,2,97.180000,13.500000,30,0,0,5,35,9266102,2,48,44,0,3,5.333333,24.000000,22.000000,10,30,20,0,60,95,0.600000,0.957001,0.312169,0.25,0.333333,0.928521,0.862500,4.243524
790,9307892,28.735453,9307892,4.740671,8.246211,1.683907e+09,3,11,15,73.333333,9307892,1.685146e+09,5,5,113,22.600000,11,0,0,3,66.346667,3.666667,0,0,0,5,5,9307892,6,114,163,0,7,9.392857,19.000000,27.166667,20,0,20,0,40,45,0.266667,0.900424,0.223210,0.50,1.000000,0.284748,0.858750,4.033798
791,9329494,NaN,9329494,NaN,NaN,1.696550e+09,1,13,14,92.857143,9329494,1.696594e+09,2,2,26,13.000000,13,0,0,1,92.860000,13.000000,30,0,0,5,35,9329494,2,24,26,0,2,9.335000,12.000000,13.000000,30,0,30,0,60,95,0.600000,NaN,0.298413,0.00,0.000000,NaN,0.918750,NaN
792,9346854,26.898605,9346854,3.358757,8.485281,1.676045e+09,3,63,44,143.181818,9346854,1.676045e+09,3,2,77,38.500000,63,0,0,3,144.973333,21.000000,40,0,0,5,45,9346854,2,52,77,0,3,8.416667,26.000000,38.500000,0,0,10,0,10,55,0.333333,0.907915,0.492256,0.50,0.333333,0.493245,0.759375,3.819458


In [50]:
final_sorted_allrounder = final_allrounder3.sort_values(by=['sum'], ascending=[False])
final_sorted_allrounder.head()

,batsman_id,std_strike_rate,bowler_id_x,std_economy,std_bowler_strike_rate,recency_score_x,matches_played_x,total_runs,total_balls_faced,recent_sr,bowler_id_y,recency_score_y,matches_played_y,total_wickets,total_runs_conceded,average_runs_conceded,cumulative_runs,half_centuries,centuries,matches_x,strike_rate,average,score_strike,score_half_centuries,score_centuries,score_average,total_score,bowler_id,cumulative_wickets,balls_bowled,runs_given,count_4w_per_inning,matches_y,economy,strike_rate_bowl,average_bowl,score_strike_bowl,score_economy,score_average_bowl,score_4w,total_score_bowl,total_score_allround,total_score_normalised,std_sr_normalised,recent_sr_normalised,recent_matches_batsman,recent_matches_bowler,std_bowler_economy_normalised,recent_average_normalised,sum
731,8246440,52.279858,8246440,1.534747,7.970315,1.695324e+09,5,67,57,117.543860,8246440,1.698123e+09,5,9,97,10.777778,114,0,0,8,105.867500,14.250000,40,0,0,5,45,8246440,23,247,214,1,12,5.320000,10.739130,9.304348,30,30,30,10,100,145,0.933333,0.804398,0.393502,1.0,1.0,0.768444,0.932639,5.832317
308,3633503,41.662773,3633503,1.134196,7.277820,1.692852e+09,5,95,99,95.959596,3633503,1.692818e+09,5,8,93,11.625000,344,1,1,17,101.850000,20.235294,40,5,10,5,60,3633503,11,174,163,0,10,5.841000,15.818182,14.818182,20,30,30,0,80,140,0.900000,0.847700,0.310363,1.0,1.0,0.828877,0.927344,5.814284
273,3298427,62.424451,3298427,1.555194,5.201398,1.687807e+09,5,98,68,144.117647,3298427,1.687236e+09,5,4,103,25.750000,762,5,0,31,111.541613,24.580645,40,20,0,5,65,3298427,23,352,399,0,23,6.833043,15.304348,17.347826,20,30,30,0,80,145,0.933333,0.763023,0.495861,1.0,1.0,0.765359,0.839063,5.796639
675,7906968,64.830608,7906968,1.705561,4.516636,1.698175e+09,5,132,96,137.500000,7906968,1.698175e+09,5,8,99,12.375000,493,2,0,18,118.297778,27.388889,40,5,0,5,50,7906968,17,264,295,0,14,6.536429,15.529412,17.352941,20,30,30,0,80,130,0.833333,0.753210,0.470370,1.0,1.0,0.742672,0.922656,5.722242
438,4739552,87.461390,4739552,2.140649,5.201343,1.693492e+09,5,80,35,228.571429,4739552,1.694598e+09,5,5,113,22.600000,573,1,0,37,124.874054,15.486486,40,5,0,5,50,4739552,63,874,1318,1,49,9.219796,13.873016,20.920635,30,0,20,10,60,110,0.700000,0.660911,0.821164,1.0,1.0,0.677028,0.858750,5.717853


In [51]:
# comparing with near best 11 provided
allrounder_best11_df =pd.merge(final_sorted_allrounder, nearbest_11, left_on='batsman_id', right_on='Player_id', how='inner')
allrounder_best11_df

,batsman_id,std_strike_rate,bowler_id_x,std_economy,std_bowler_strike_rate,recency_score_x,matches_played_x,total_runs,total_balls_faced,recent_sr,bowler_id_y,recency_score_y,matches_played_y,total_wickets,total_runs_conceded,average_runs_conceded,cumulative_runs,half_centuries,centuries,matches_x,strike_rate,average,score_strike,score_half_centuries,score_centuries,score_average,total_score,bowler_id,cumulative_wickets,balls_bowled,runs_given,count_4w_per_inning,matches_y,economy,strike_rate_bowl,average_bowl,score_strike_bowl,score_economy,score_average_bowl,score_4w,total_score_bowl,total_score_allround,total_score_normalised,std_sr_normalised,recent_sr_normalised,recent_matches_batsman,recent_matches_bowler,std_bowler_economy_normalised,recent_average_normalised,sum,Player_id
0,4171460,43.931313,4171460,1.200602,8.335030,1.690917e+09,5,90,55,163.636364,4171460,1.690554e+09,5,5,140,28.0,400,0,0,27,103.976667,14.814815,40,0,0,5,45,4171460,21,462,631,1,26,8.708462,22.0,30.047619,10,0,10,10,30,75,0.466667,0.838447,0.571044,1.0,1.0,0.818858,0.8250,5.520016,4171460
1,2275195,75.142067,2275195,3.386464,6.131763,1.702668e+09,5,191,99,192.929293,2275195,1.699678e+09,5,5,146,29.2,2480,17,3,95,126.458211,26.105263,40,20,30,5,95,2275195,36,882,1154,0,68,8.365588,24.5,32.055556,0,0,10,0,10,105,0.666667,0.711155,0.683876,1.0,1.0,0.489065,0.8175,5.368262,2275195


# ADDING BEST 11 PLAYERS TO FILE

In [52]:
my11_batsman2 = final_sorted.head(3)['batsman_id']
my11_batsman = final_sorted.head(3).reset_index(drop=True)
final_sorted.head(3)

,batsman_id,std_strike_rate,recency_score,matches_played,total_runs,total_balls_faced,recent_sr,cumulative_runs,half_centuries,centuries,matches,strike_rate,average,score_strike,score_half_centuries,score_centuries,score_average,total_score,total_score_normalised,std_sr_normalised,recent_sr_normalised,recent_matches_played,sum
183,2275195,75.142067,1.702668e+09,5,191,99,192.929293,2480,17,3,95,126.458211,26.105263,40,20,30,5,95,0.947368,0.644981,0.828428,1.0,3.420777
164,2162782,58.786308,1.697103e+09,5,233,149,156.375839,3498,32,6,97,123.307835,36.061856,40,20,30,10,100,1.000000,0.739542,0.652468,1.0,3.392010
180,2263736,43.395586,1.680670e+09,5,179,134,133.582090,4031,40,2,92,115.673261,43.815217,40,20,20,20,100,1.000000,0.828523,0.542745,1.0,3.371268


In [53]:
my11_wicketkeeper = final_sorted_wk[~final_sorted_wk['batsman_id'].isin(my11_batsman2)]
my11_wicketkeeper.head(1)

,batsman_id,std_strike_rate,recency_score,matches_played,total_runs,total_balls_faced,recent_sr,cumulative_runs,half_centuries,centuries,matches,strike_rate,is_batsman_keeper,average,score_strike,score_half_centuries,score_centuries,score_average,total_score,total_score_normalised,std_sr_normalised,recent_sr_normalised,recent_matches_played,sum
108,4685568,60.998638,1.696861e+09,5,249,122,204.098361,2525,15,1,105,132.002952,63,24.047619,40,20,10,5,75,0.736842,0.427675,0.879994,1.0,3.044512


In [54]:
my11_bowler2 = final_sorted_bowler.head(3)['bowler_id']
my11_bowler = final_sorted_bowler.head(3).reset_index(drop=True)
final_sorted_bowler.head(3)

,bowler_id,std_economy,std_bowler_strike_rate,recency_score,matches_played,total_wickets,total_runs_conceded,average_runs_conceded,cumulative_wickets,balls_bowled,runs_given,count_4w_per_inning,matches,economy,strike_rate_bowl,average_bowl,score_strike_bowl,score_economy,score_average_bowl,score_4w,total_score_bowl,total_score_bowler_normalised,std_bowler_economy_normalised,recent_average_normalised,sum
463,8246440,1.534747,7.970315,1.698123e+09,5,9,97,10.777778,23,247,214,1,12,5.320,10.739130,9.304348,30,30,30,10,100,1.0,0.830737,0.932639,2.763375
202,3633503,1.134196,7.277820,1.692818e+09,5,8,93,11.625000,11,174,163,0,10,5.841,15.818182,14.818182,20,30,30,0,80,0.8,0.936345,0.927344,2.663688
464,8246573,1.832357,6.231401,1.689708e+09,5,10,125,12.500000,14,151,167,0,7,6.820,10.785714,11.928571,30,30,30,0,90,0.9,0.752270,0.921875,2.574145


In [55]:
my11_allrounder = final_sorted_allrounder[~final_sorted_allrounder['batsman_id'].isin(my11_batsman2)]
my11_allrounder2 = my11_allrounder[~my11_allrounder['bowler_id'].isin(my11_bowler2)]
my11_allrounder2.head(4)

,batsman_id,std_strike_rate,bowler_id_x,std_economy,std_bowler_strike_rate,recency_score_x,matches_played_x,total_runs,total_balls_faced,recent_sr,bowler_id_y,recency_score_y,matches_played_y,total_wickets,total_runs_conceded,average_runs_conceded,cumulative_runs,half_centuries,centuries,matches_x,strike_rate,average,score_strike,score_half_centuries,score_centuries,score_average,total_score,bowler_id,cumulative_wickets,balls_bowled,runs_given,count_4w_per_inning,matches_y,economy,strike_rate_bowl,average_bowl,score_strike_bowl,score_economy,score_average_bowl,score_4w,total_score_bowl,total_score_allround,total_score_normalised,std_sr_normalised,recent_sr_normalised,recent_matches_batsman,recent_matches_bowler,std_bowler_economy_normalised,recent_average_normalised,sum
273,3298427,62.424451,3298427,1.555194,5.201398,1.687807e+09,5,98,68,144.117647,3298427,1.687236e+09,5,4,103,25.750,762,5,0,31,111.541613,24.580645,40,20,0,5,65,3298427,23,352,399,0,23,6.833043,15.304348,17.347826,20,30,30,0,80,145,0.933333,0.763023,0.495861,1.0,1.0,0.765359,0.839063,5.796639
675,7906968,64.830608,7906968,1.705561,4.516636,1.698175e+09,5,132,96,137.500000,7906968,1.698175e+09,5,8,99,12.375,493,2,0,18,118.297778,27.388889,40,5,0,5,50,7906968,17,264,295,0,14,6.536429,15.529412,17.352941,20,30,30,0,80,130,0.833333,0.753210,0.470370,1.0,1.0,0.742672,0.922656,5.722242
438,4739552,87.461390,4739552,2.140649,5.201343,1.693492e+09,5,80,35,228.571429,4739552,1.694598e+09,5,5,113,22.600,573,1,0,37,124.874054,15.486486,40,5,0,5,50,4739552,63,874,1318,1,49,9.219796,13.873016,20.920635,30,0,20,10,60,110,0.700000,0.660911,0.821164,1.0,1.0,0.677028,0.858750,5.717853
536,6282856,69.010822,6282856,1.895994,7.250616,1.697397e+09,5,80,49,163.265306,6282856,1.688964e+09,5,5,82,16.400,2150,17,1,80,131.639250,26.875000,40,20,10,5,75,6282856,21,340,410,0,30,7.833000,16.190476,19.523810,20,0,30,0,50,125,0.800000,0.736161,0.569615,1.0,1.0,0.713941,0.897500,5.717216


In [56]:
nearbest_11

,Player_id
0,325814
1,1626526
2,1506098
3,82228
4,8250115
5,4171460
6,2275195
7,5652758
8,8986193
9,8246573


In [57]:
my11_pandas = pd.read_csv("my11_from_pandas2.csv")
my11_pandas.head()

,player_id,total_points,role,runs,batting_strike_rate,batting_average,100s,50s,wickets,bowling_strike_rate,economy,bowling_average,4w_per_innings


In [58]:
batsman_appending = my11_batsman[['batsman_id', 'total_score', 'cumulative_runs', 'strike_rate', 'average', 'centuries', 'half_centuries']]
batsman_appending.columns = ['player_id', 'total_points', 'runs', 'batting_strike_rate', 'batting_average', '100s', '50s']

batsman_appending['role'] = 'batsman'

bat_my11_pandas = pd.concat([my11_pandas, batsman_appending], ignore_index=True)
bat_my11_pandas

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_13712\961984223.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batsman_appending['role'] = 'batsman'


,player_id,total_points,role,runs,batting_strike_rate,batting_average,100s,50s,wickets,bowling_strike_rate,economy,bowling_average,4w_per_innings
0,2275195,95,batsman,2480,126.458211,26.105263,3,17,NaN,NaN,NaN,NaN,NaN
1,2162782,100,batsman,3498,123.307835,36.061856,6,32,NaN,NaN,NaN,NaN,NaN
2,2263736,100,batsman,4031,115.673261,43.815217,2,40,NaN,NaN,NaN,NaN,NaN


In [59]:
wk_appending = my11_wicketkeeper.head(1)[['batsman_id', 'total_score', 'cumulative_runs', 'strike_rate', 'average', 'centuries', 'half_centuries']]
wk_appending.columns = ['player_id', 'total_points', 'runs', 'batting_strike_rate', 'batting_average', '100s', '50s']

wk_appending['role'] = 'wicketkeeper'

wk_my11_pandas = pd.concat([bat_my11_pandas, wk_appending], ignore_index=True)
wk_my11_pandas

,player_id,total_points,role,runs,batting_strike_rate,batting_average,100s,50s,wickets,bowling_strike_rate,economy,bowling_average,4w_per_innings
0,2275195,95,batsman,2480,126.458211,26.105263,3,17,NaN,NaN,NaN,NaN,NaN
1,2162782,100,batsman,3498,123.307835,36.061856,6,32,NaN,NaN,NaN,NaN,NaN
2,2263736,100,batsman,4031,115.673261,43.815217,2,40,NaN,NaN,NaN,NaN,NaN
3,4685568,75,wicketkeeper,2525,132.002952,24.047619,1,15,NaN,NaN,NaN,NaN,NaN


In [60]:
all_appending = my11_allrounder2.head(4)[['batsman_id', 'total_score_allround', 'cumulative_runs', 'strike_rate', 'average', 'centuries', 'half_centuries', 'cumulative_wickets', 'strike_rate_bowl', 'economy', 'average_bowl', 'count_4w_per_inning']]
all_appending.columns = ['player_id', 'total_points', 'runs', 'batting_strike_rate', 'batting_average', '100s', '50s','wickets', 'bowling_strike_rate', 'economy', 'bowling_average', '4w_per_innings']

all_appending['role'] = 'all-rounder'

all_my11_pandas = pd.concat([wk_my11_pandas, all_appending], ignore_index=True)
all_my11_pandas

,player_id,total_points,role,runs,batting_strike_rate,batting_average,100s,50s,wickets,bowling_strike_rate,economy,bowling_average,4w_per_innings
0,2275195,95,batsman,2480,126.458211,26.105263,3,17,NaN,NaN,NaN,NaN,NaN
1,2162782,100,batsman,3498,123.307835,36.061856,6,32,NaN,NaN,NaN,NaN,NaN
2,2263736,100,batsman,4031,115.673261,43.815217,2,40,NaN,NaN,NaN,NaN,NaN
3,4685568,75,wicketkeeper,2525,132.002952,24.047619,1,15,NaN,NaN,NaN,NaN,NaN
4,3298427,145,all-rounder,762,111.541613,24.580645,0,5,23,15.304348,6.833043,17.347826,0
5,7906968,130,all-rounder,493,118.297778,27.388889,0,2,17,15.529412,6.536429,17.352941,0
6,4739552,110,all-rounder,573,124.874054,15.486486,0,1,63,13.873016,9.219796,20.920635,1
7,6282856,125,all-rounder,2150,131.639250,26.875000,1,17,21,16.190476,7.833000,19.523810,0


In [61]:
ball_appending = my11_bowler.head(3)[['bowler_id', 'total_score_bowl', 'cumulative_wickets', 'strike_rate_bowl', 'economy', 'average_bowl', 'count_4w_per_inning']]
ball_appending.columns = ['player_id', 'total_points','wickets', 'bowling_strike_rate', 'economy', 'bowling_average', '4w_per_innings']

ball_appending['role'] = 'bowler'

final_my11_pandas = pd.concat([all_my11_pandas, ball_appending], ignore_index=True)
final_my11_pandas

,player_id,total_points,role,runs,batting_strike_rate,batting_average,100s,50s,wickets,bowling_strike_rate,economy,bowling_average,4w_per_innings
0,2275195,95,batsman,2480,126.458211,26.105263,3,17,NaN,NaN,NaN,NaN,NaN
1,2162782,100,batsman,3498,123.307835,36.061856,6,32,NaN,NaN,NaN,NaN,NaN
2,2263736,100,batsman,4031,115.673261,43.815217,2,40,NaN,NaN,NaN,NaN,NaN
3,4685568,75,wicketkeeper,2525,132.002952,24.047619,1,15,NaN,NaN,NaN,NaN,NaN
4,3298427,145,all-rounder,762,111.541613,24.580645,0,5,23,15.304348,6.833043,17.347826,0
5,7906968,130,all-rounder,493,118.297778,27.388889,0,2,17,15.529412,6.536429,17.352941,0
6,4739552,110,all-rounder,573,124.874054,15.486486,0,1,63,13.873016,9.219796,20.920635,1
7,6282856,125,all-rounder,2150,131.639250,26.875000,1,17,21,16.190476,7.833000,19.523810,0
8,8246440,100,bowler,NaN,NaN,NaN,NaN,NaN,23,10.739130,5.320000,9.304348,1
9,3633503,80,bowler,NaN,NaN,NaN,NaN,NaN,11,15.818182,5.841000,14.818182,0


In [62]:
final_my11_pandas.to_csv('my11_from_pandas2.csv', index=False, encoding='utf-8')